In [12]:

from fastapi import HTTPException
from typing import Any, Dict, Optional
import json
import hmac
import hashlib
import httpx

AGENT_SECRET = 'dEcartes2026'
IP = '136.111.38.85'

def json_dumps(payload: dict) -> bytes:
    return json.dumps(payload, separators=(",", ":"), sort_keys=True).encode()


def call_agent_hmac(ip, path, secret=AGENT_SECRET, json_body=None, method="POST", timeout=5):
    url = f"http://{ip}:8000{path}"
    
    if method == "DELETE":
        # Use path param (fqdn) as body for HMAC
        fqdn = path.split("/")[-1]
        body_bytes = fqdn.encode()
        signature = hmac.new(secret.encode(), body_bytes, hashlib.sha256).hexdigest()
        headers = {
            "Content-Type": "text/plain",
            "X-Signature": signature,
        }
        res = httpx.request("DELETE", url, content=body_bytes, headers=headers, timeout=timeout)

    else:
        headers = {"Content-Type": "application/json"}
        body_bytes = json_dumps(json_body) if json_body else b"{}"
        signature = hmac.new(secret.encode(), body_bytes, hashlib.sha256).hexdigest()
        headers["X-Signature"] = signature
        
        if method == "POST":
            res = httpx.post(url, content=body_bytes, headers=headers, timeout=timeout)
        elif method == "GET":
            body_bytes = b""
            signature = hmac.new(secret.encode(), body_bytes, hashlib.sha256).hexdigest()
            headers = {"X-Signature": signature}
            res = httpx.get(url, headers=headers, timeout=timeout)

    try:
        res.raise_for_status()
    except httpx.HTTPStatusError as e:
        raise HTTPException(
            status_code=e.response.status_code,
            detail=f"Agent error: {e.response.text}"
        )
    return res


In [68]:
# Load JSON from file
with open("../json/dnsproof.org.json", "r") as f:
    payload = json.load(f)

# Send request
response = call_agent_hmac(
    ip=IP,
    path="/internal/dns/push",
    json_body=payload
)

print(response.json())

{'status': 'success'}


#### DNSSEC enabling

In [55]:
# Send request
response = call_agent_hmac(
    ip=IP,
    path="/internal/dns/dnssec/sign/dnsproof.org"
)

print(response.json())

{'rrsig_records': ['SOA 8 2 3600 20260209083539 20260112083539 8689 @ jJasonvyjqXi/Qu1HlidOMiWIkOWIn5w xAmi2mWbecUepqwrr82HJbyvaBqCEPoV +tnknbSvPBnxh67sjnZcFYsLq7zFsJxa 9MPT7he0acak70lqUeNfgA8y7uiti9Ft gcsyt6oeWz7lZVjXKPqNVl2LD1tPqJdq Wb297uqh3Es=', 'NS 8 2 3600 20260209083539 20260112083539 8689 @ W97+1t0qHWTP1bN3LsnbXI/SBCRRNTbd TvCHX5Qv49DjEvWGfAqtTDi8/mxkWZaA aHsuorNY0aZbdxhM9uNqnul5VoBkuCmg qQnRccrJlUt3t8cRLtUkyuagLLVKlRYe oN6Lfygeqd3UpauwjnUnZ9fs9m1GtUIq 98Ex1pUKlA8=', 'MX 8 2 3600 20260209083539 20260112083539 8689 @ vc3qvIlkR46QoXi6EpqLff06pamxO8oX L8/qb6gFeEIzy1T7VIKz5cXKIJFd9rbX AQRiPiy+tdDqFXRppNcWLH/lBlHrsxwT lqNOx2z6zhE/eQkjN9H22mmjhDt+gja8 +/OY3ukdOvUuk5cBwkeP7zs/q3yaHQpO wXT2YefWsDo=', 'TXT 8 2 3600 20260209083539 20260112083539 8689 @ ShHRylxiMU3KmuqbK+Ufx6SyPN6bkk3P vCtCjfPShevb5GmUF0YqGYcJZ5jBgJyv qM76KgAASWnKt5irEat//ybFDfOOHDCc LYRzjvWjKAkBEK+S7JQ7yEhg+fjWLyZg LG6y1/NVZPDcw7BPXZa+pF0XEaZXVx0M GpD9JbBzJCk=', 'DNSKEY 8 2 3600 20260209083539 20260112083539 60209 @ h8Ue

#### DNSSEC disabling

In [54]:
# Send request
response = call_agent_hmac(
    ip=IP,
    path="/internal/dns/dnssec/disable/dnsproof.org"
)

print(response.json())

{'status': 'success'}


#### DNSSEC status

In [63]:
response = call_agent_hmac(
    ip=IP,
    path="/internal/dns/dnssec/status/dnsproof.org",
    method="GET"
)

print(response.json())

{'ksk_created_at': '2026-01-12T08:46:27.613463', 'zsk_created_at': '2026-01-12T08:46:27.597461', 'ds_digest': '64F5B049589AC5D84F69B7EFC3B78FD8A5E59C47B405A027771199CFC70EC98A', 'key_tag': 55149, 'algorithm': 8, 'days_since_last_key_creation': 0}


#### DNSSEC key rotation

In [69]:
response = call_agent_hmac(
    ip=IP,
    path="/internal/dns/dnssec/rotate/dnsproof.org"
)

print(response.json())

{'rrsig_records': ['SOA 8 2 3600 20260209090924 20260112090924 14349 @ X8CEngj59Dh9aqEwm+eo6immLKUw8Czx pcFEisSOt2EziH50CH5yNUbYR/5DJ6Pg 5DUhW55pr0ADnUZn89MqroDJbRvZJ6Q6 eShK0clUFwHWwo0CTUjQsh+/BacpUMwl 752H4vd+Vycn2wUgjOKVVgk/i6J3q3tr uu9UeJ6ealY=', 'NS 8 2 3600 20260209090924 20260112090924 14349 @ pfmFk4tuoDhyLbMGV4TDyiBh39RO5dPK jsSpa5EUVNUuBD6ULyVLfxUJ3T9VBWWW J3GZlQMc2V027xSA0DZBBPwcl2KgiekN ta5uoovbiU2vma4dihp8NPgJGF/10l4f IqViI1JM6wKf/hi6qpVUerlD2e2VPr7g 4KTcg3DzN88=', 'MX 8 2 3600 20260209090924 20260112090924 14349 @ jfEEIWG2vS+t/g397kVxD3QmuX7HjUyk DP4GsXTPkzrCTfnVyypavYWKcF/4A/FZ WQvu+5AQk0DyzRc08+gHi8BaesYd85pV p8syka/+Vhl3R25WT9FXD/JVr6P4u5LJ g+FUp8V/K830T2gFtvOBhWih6ACH/uS6 Yd/6BCV1X1E=', 'TXT 8 2 3600 20260209090924 20260112090924 14349 @ de6xatrK6sitbHTdDFeU0V76hp89H81o nk8YjLsGEn/9DXTU5QDjVG257yZk+PMk LRwF1PmclEACKEGqLY16lxnRPxVNL7mz jPy0xnBQmP36le5ftTB3Z2I7NhEm2MBK sBaoCECmEeG+M+AwirJYKe8BWiTba1zj Xi8CHiMAa98=', 'DNSKEY 8 2 3600 20260209090924 20260112090924 6706 @ B